In [15]:
import pandas as pd
import requests

from config import api_key
from config import secret_key


In [16]:
# see https://curl.trillworks.com/ for curl --> python
# see https://www.petfinder.com/developers/v2/docs/ for API

access_token_query_url = "https://api.petfinder.com/v2/oauth2/token"

parameters = {
  'grant_type': 'client_credentials',
  'client_id': api_key,
  'client_secret': secret_key
}

access_token = requests.post(access_token_query_url, data=parameters).json()["access_token"]


In [17]:
# curl -H "Authorization: Bearer {YOUR_ACCESS_TOKEN}" GET https://api.petfinder.com/v2/{CATEGORY}/{ACTION}?{parameter_1}={value_1}&{parameter_2}={value_2}

# category = animals | types | organizations
# parameters = type | breed | size | gender | age | color | coat | status | name | organization | location | distance | sort | page | limit

#########    Option 1 - by city name    ############################
parameter1 = "?type=dog"
parameter2 = "&limit=10"
parameter3 = "&location=Atlanta, GA"
data_query_url = "https://api.petfinder.com/v2/animals" + parameter1 + parameter2 + parameter3

headers = {
    'Authorization': 'Bearer {}'.format(access_token),
}

response = requests.get(data_query_url, headers=headers).json() 
response   

{'animals': [{'id': 44293601,
   'organization_id': 'GA929',
   'url': 'https://www.petfinder.com/dog/micki-44293601/ga/snellville/georgia-akita-rescue-division-ga929/?referrer_id=4b9f6d7d-e6a4-46e5-ae76-dab6e06a6400',
   'type': 'Dog',
   'species': 'Dog',
   'breeds': {'primary': 'Akita',
    'secondary': None,
    'mixed': False,
    'unknown': False},
   'colors': {'primary': 'Black',
    'secondary': 'White / Cream',
    'tertiary': None},
   'age': 'Adult',
   'gender': 'Female',
   'size': 'Extra Large',
   'coat': None,
   'attributes': {'spayed_neutered': True,
    'house_trained': True,
    'declawed': None,
    'special_needs': False,
    'shots_current': True},
   'environment': {'children': None, 'dogs': False, 'cats': False},
   'tags': [],
   'name': 'Micki',
   'description': 'Micki is a beautiful 7 year old female who comes to us from a shelter in WV. Micki spent her...',
   'photos': [{'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/44293601/1/?bust=1553301

In [15]:
# save response to json file

#import json
#with open("response.json", "w") as json_file:
#    json.dump(response, json_file)

In [59]:
# save dataFrame to mongoDB
#import pymongo

#mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")

# mongoDB db name = pets
#mongo_pet_db = mongo_client['pets'] 
#mongoDB collection name = pets_for_you
#mongo_pet_collection = mongo_pet_db["pets_for_you"]


#mongo_pet_collection.remove()
#mongo_pet_collection.insert(response["animals"])

############# end of option 1 ########################

In [20]:
################### Option 2 - by pages###########################

import time 
import pymongo

data_query_url_base = "https://api.petfinder.com/v2/animals?type=dog&limit=100"

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
# mongoDB db name = pets
mongo_pet_db = mongo_client['pets'] 
#mongoDB collection name = pets_for_you
mongo_pet_collection = mongo_pet_db["pets_for_you"]


for i in range(11):
  page_parameter = "&page=" + str(i+1)
  data_query_url = data_query_url_base + page_parameter
  
  headers = {
    'Authorization': 'Bearer {}'.format(access_token),
  }

  response = requests.get(data_query_url, headers=headers).json()

  mongo_pet_collection.insert(response["animals"])
  
  time.sleep(2)
  

###### end of option 2 ######################

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [21]:
# You need clean up json file to save to pandas
import pandas as pd
import json

data = response["animals"]

url = []
species = []
breed = []
color = []
age = []
gender = []
size = []
coat = []
neutered = []
house_trained = []
declawed = []
special_needs = []
shots = []
name = []
description = []
photo = []
status = []
publish_date = []
contact_email = []
contact_phone = []
contact_address1 = []
contact_address2 = []
contact_city = []
contact_state = []
contact_postcode = []
contact_country = []

for item in data:
    url.append(item["url"])
    species.append(item["species"])
    breed.append(item["breeds"]["primary"])
    color.append(item["colors"]["primary"])
    age.append(item["age"])
    gender.append(item["gender"])
    size.append(item["size"])
    coat.append(item["coat"])
    neutered.append(item["attributes"]["spayed_neutered"])
    house_trained.append(item["attributes"]["house_trained"])
    declawed.append(item["attributes"]["declawed"])
    special_needs.append(item["attributes"]["special_needs"])
    shots.append(item["attributes"]["shots_current"])
    name.append(item["name"])
    description.append(item["description"])
    
    if len(item["photos"]) != 0:
        photo.append(item["photos"][0]["full"])
    else:
        photo.append("NULL")

    status.append(item["status"])
    publish_date.append(item["published_at"])
    contact_email.append(item["contact"]["email"])
    contact_phone.append(item["contact"]["phone"])
    contact_address1.append(item["contact"]["address"]["address1"])
    contact_address2.append(item["contact"]["address"]["address2"])
    contact_city.append(item["contact"]["address"]["city"])
    contact_state.append(item["contact"]["address"]["state"])
    contact_postcode.append(item["contact"]["address"]["postcode"])
    contact_country.append(item["contact"]["address"]["country"])
    


data_dict = {
    "url" : url,
    "species" : species,
    "breed" : breed,
    "color" : color,
    "age" : age,
    "gender" : gender,
    "size" : size,
    "coat" : coat,
    "neutered" : neutered,
    "house_trained" : house_trained,
    "declawed" : declawed,
    "special_needs" : special_needs,
    "shots" : shots,
    "name" : name,
    "description" : description,
    "photo" : photo,
    "status" : status,
    "publish_date" : publish_date,
    "contact_email" : contact_email,
    "contact_phone" : contact_phone,
    "contact_address1" : contact_address1,
    "contact_address2" : contact_address2,
    "contact_city" : contact_city,
    "contact_state" : contact_state,
    "contact_postcode" : contact_postcode,
    "contact_country" : contact_country
}

#store data to pandas
df_petData = pd.DataFrame(data = data_dict)
#df_petData

# to csv
#df_petData.to_csv(output_data_file, index=False)

# to mongoDB
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
# mongoDB db name = pets
mongo_pet_db = mongo_client['pets'] 
#mongoDB collection name = pets_for_you
mongo_pet_collection = mongo_pet_db["pets_for_you"]

records = json.loads(df_petData.T.to_json()).values()
mongo_pet_collection.insert(records)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:113: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


[ObjectId('5c9587097cb0174504dd1b79'),
 ObjectId('5c9587097cb0174504dd1b7a'),
 ObjectId('5c9587097cb0174504dd1b7b'),
 ObjectId('5c9587097cb0174504dd1b7c'),
 ObjectId('5c9587097cb0174504dd1b7d'),
 ObjectId('5c9587097cb0174504dd1b7e'),
 ObjectId('5c9587097cb0174504dd1b7f'),
 ObjectId('5c9587097cb0174504dd1b80'),
 ObjectId('5c9587097cb0174504dd1b81'),
 ObjectId('5c9587097cb0174504dd1b82'),
 ObjectId('5c9587097cb0174504dd1b83'),
 ObjectId('5c9587097cb0174504dd1b84'),
 ObjectId('5c9587097cb0174504dd1b85'),
 ObjectId('5c9587097cb0174504dd1b86'),
 ObjectId('5c9587097cb0174504dd1b87'),
 ObjectId('5c9587097cb0174504dd1b88'),
 ObjectId('5c9587097cb0174504dd1b89'),
 ObjectId('5c9587097cb0174504dd1b8a'),
 ObjectId('5c9587097cb0174504dd1b8b'),
 ObjectId('5c9587097cb0174504dd1b8c'),
 ObjectId('5c9587097cb0174504dd1b8d'),
 ObjectId('5c9587097cb0174504dd1b8e'),
 ObjectId('5c9587097cb0174504dd1b8f'),
 ObjectId('5c9587097cb0174504dd1b90'),
 ObjectId('5c9587097cb0174504dd1b91'),
 ObjectId('5c9587097cb017